# Lyrics to Artist

We train a neural network on lyrics from 10 different artists. Each artist has 100 songs to train on, and then we test the network

### Libraries, API, and constants setup

In [33]:
import lyricsgenius as genius
import os
import json
from tqdm import tqdm
import matplotlib.pylab as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

# Duncan's API key, change to your own if you are downloading data
geniusCreds = 'Lw6NjXtbU7NndUFHRCcOX9FdLhPzVokLIt9c4LWzsTxM10wF7EICGtWSSso8Ohsq'
NUM_OF_SONGS = 100

artist_names = [
#     'James Taylor',
#     'The Beatles',
#     'Johnny Cash',
#     'Van Morrison',
#     'Bob Dylan',
#     'Fleetwood Mac',
#     'Lil Wayne',
#     'J. Cole',
    'Taylor Swift',
    'Beyonce'
]

api = genius.Genius(geniusCreds)
genius.skip_non_songs = True
genius.excluded_terms = ["(Remix)", "(Live)"]

### Download Lyric Data

In [34]:
for artist_name in artist_names:
    artist = api.search_artist(artist_name, max_songs=NUM_OF_SONGS, sort='Title')
    artist.save_lyrics()

Searching for songs by Taylor Swift...



TypeError: 'NoneType' object is not subscriptable

### Clean the Data
Remove excess JSON data and write to a new file, just with the lyrics

In [11]:
for artist in tqdm(artist_names):
    a_mod = artist.replace(' ', '')
    filename = f'Lyrics_{a_mod}.json'
    with open(f'./data/{filename}') as f:
        data = json.load(f)
        song_obj = {}
        song_obj[artist] = []
        for song in data['songs']:
            song_obj[artist].append(song['lyrics'])
        with open(f'./data/cleaned/{artist}.json', 'w') as outfile:
            json.dump(song_obj, outfile)

100%|██████████| 1/1 [00:00<00:00, 22.51it/s]


### Extract, tokenize, and one hot encode data to X, Y vectors

In [19]:
# get basic X, Y sets for training + testing
# X is song lyrics
# Y is the one-hot encoding of the artists
X = []
Y = []
NUM_OF_ARTISTS = len(list(os.listdir('./data/cleaned'))) - 1

index = 0
for filename in os.listdir('./data/cleaned'):
    with open(f'./data/cleaned/{filename}') as file:
        if filename == '.DS_Store': continue
        data = json.load(file)[filename.replace('.json', '')]
        if len(data) == 100:
            for song in data:
                X.append(song)
                # one hot encode artist
                one_h = np.zeros(NUM_OF_ARTISTS)
                one_h[index] = 1
                Y.append(one_h)
            index += 1

In [20]:
# create encoding of words
# i.e. map words to numbers
big_word_list = {}
for song in X:
    for word in song.split(' '):
        if word not in big_word_list.keys():
            if (len(big_word_list.keys()) == 0):
                big_word_list[word] = 1
            else:
                big_word_list[word] = len(big_word_list) + 1
                
# tokenize the songs
# so we are mapping each word to it's key in the big_word_list
token_x = []
for song in X:
    new_song = []
    for word in song.split(' '):
        new_song.append(big_word_list[word])
    token_x.append(np.array(new_song))
X = token_x

# cutting off lyrics min 100 words
# to normalize network input
cutoff = 100
index = 0
new_x = []
new_y = []
for song in X:
    if (len(song) >= cutoff):
        new_x.append(song)
        new_y.append(Y[index])
    index += 1

X = new_x
Y = new_y

# trim all of the lyrics to 100 words max,
# so the network input in normalized
trimmed = []
for song in X:
    song = song[:100]
    trimmed.append(song)
X = trimmed

# convert to numpy arrays for keras
X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

(740, 100)
(740, 8)


### Create model, train and evaluate

In [24]:
model = Sequential([
    Dense(128, input_shape=(100,)),
    Activation('softmax'),
    Dense(1500),
    Activation('softmax'),
    Dense(500),
    Activation('relu'),
    Dense(225),
    Activation('relu'),
    Dense(64),
    Activation('softmax'),
    Dense(NUM_OF_ARTISTS)
])

model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

In [25]:
model.fit(X, Y, epochs=250, batch_size=24)

Epoch 1/250
740/740 [==============================] - 1s 791us/step - loss: 0.1178 - accuracy: 0.1311
Epoch 2/250
740/740 [==============================] - 0s 494us/step - loss: 0.1097 - accuracy: 0.1297
Epoch 3/250
740/740 [==============================] - 0s 488us/step - loss: 0.1094 - accuracy: 0.1162
Epoch 4/250
740/740 [==============================] - 0s 487us/step - loss: 0.1094 - accuracy: 0.1257
Epoch 5/250
740/740 [==============================] - 0s 487us/step - loss: 0.1095 - accuracy: 0.1203
Epoch 6/250
740/740 [==============================] - 0s 485us/step - loss: 0.1093 - accuracy: 0.1297
Epoch 7/250
740/740 [==============================] - 0s 482us/step - loss: 0.1095 - accuracy: 0.1149
Epoch 8/250
740/740 [==============================] - 0s 489us/step - loss: 0.1095 - accuracy: 0.1162
Epoch 9/250
740/740 [==============================] - 0s 484us/step - loss: 0.1094 - accuracy: 0.1014
Epoch 10/250
740/740 [==============================] - 0s 486us/step - l

740/740 [==============================] - 0s 491us/step - loss: 0.1075 - accuracy: 0.1770
Epoch 80/250
740/740 [==============================] - 0s 585us/step - loss: 0.1074 - accuracy: 0.1932
Epoch 81/250
740/740 [==============================] - 0s 523us/step - loss: 0.1074 - accuracy: 0.1757
Epoch 82/250
740/740 [==============================] - 0s 516us/step - loss: 0.1075 - accuracy: 0.1878
Epoch 83/250
740/740 [==============================] - 0s 517us/step - loss: 0.1074 - accuracy: 0.1946
Epoch 84/250
740/740 [==============================] - 0s 505us/step - loss: 0.1074 - accuracy: 0.1905
Epoch 85/250
740/740 [==============================] - 0s 536us/step - loss: 0.1074 - accuracy: 0.1946
Epoch 86/250
740/740 [==============================] - 0s 508us/step - loss: 0.1073 - accuracy: 0.1811
Epoch 87/250
740/740 [==============================] - 0s 531us/step - loss: 0.1073 - accuracy: 0.1878
Epoch 88/250
740/740 [==============================] - 0s 500us/step - loss:

740/740 [==============================] - 0s 485us/step - loss: 0.1062 - accuracy: 0.2270
Epoch 158/250
740/740 [==============================] - 0s 488us/step - loss: 0.1065 - accuracy: 0.2162
Epoch 159/250
740/740 [==============================] - 0s 497us/step - loss: 0.1069 - accuracy: 0.2176
Epoch 160/250
740/740 [==============================] - 0s 485us/step - loss: 0.1067 - accuracy: 0.2162
Epoch 161/250
740/740 [==============================] - 0s 491us/step - loss: 0.1066 - accuracy: 0.2081
Epoch 162/250
740/740 [==============================] - 0s 535us/step - loss: 0.1065 - accuracy: 0.2135
Epoch 163/250
740/740 [==============================] - 0s 512us/step - loss: 0.1065 - accuracy: 0.2122
Epoch 164/250
740/740 [==============================] - 0s 529us/step - loss: 0.1064 - accuracy: 0.2162
Epoch 165/250
740/740 [==============================] - 0s 522us/step - loss: 0.1064 - accuracy: 0.2041
Epoch 166/250
740/740 [==============================] - 0s 504us/ste

740/740 [==============================] - 0s 493us/step - loss: 0.1051 - accuracy: 0.2135
Epoch 236/250
740/740 [==============================] - 0s 511us/step - loss: 0.1052 - accuracy: 0.2203
Epoch 237/250
740/740 [==============================] - 0s 503us/step - loss: 0.1052 - accuracy: 0.2203
Epoch 238/250
740/740 [==============================] - 0s 491us/step - loss: 0.1051 - accuracy: 0.2108
Epoch 239/250
740/740 [==============================] - 0s 497us/step - loss: 0.1053 - accuracy: 0.2149
Epoch 240/250
740/740 [==============================] - 0s 499us/step - loss: 0.1052 - accuracy: 0.2162
Epoch 241/250
740/740 [==============================] - 0s 493us/step - loss: 0.1051 - accuracy: 0.2203
Epoch 242/250
740/740 [==============================] - 0s 518us/step - loss: 0.1051 - accuracy: 0.2108
Epoch 243/250
740/740 [==============================] - 0s 492us/step - loss: 0.1050 - accuracy: 0.2230
Epoch 244/250
740/740 [==============================] - 0s 496us/ste

In [26]:
model.evaluate(X, Y)

740/740 [==============================] - 0s 114us/step


[0.10487923412709622, 0.2243243306875229]